<a href="https://colab.research.google.com/github/winifred-kay/Sentiment-Analysis-with-huggingface-fac/blob/master/FinetuningModel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.2 MB/s eta 0:00:00


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00


In [3]:

import tensorflow as tf
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import IntervalStrategy, TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
import os
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


**Sentiment Analysis using Hugging Face**

This project uses the knowledge of natural language processing and transfer learning to fine tune a model using the following process

1. Load dataset into your notebook
2. Preprocess data
3. perform training
4. Evaluate mode

In [5]:
df = pd.read_csv("/content/drive/MyDrive/data/Train.csv")

In [6]:
df.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [8]:
data = df[~df.isna().any(axis=1)]

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   9999 non-null   object 
 1   safe_text  9999 non-null   object 
 2   label      9999 non-null   float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 390.6+ KB


**Preprocessing data**

In [10]:
# Split the train data 
train, eval = train_test_split(data, test_size=0.2, random_state=30, stratify=data['label'])

In [11]:
print(f"train dim: {train.shape}, eval dim:  {eval.shape}")

train dim: (7999, 4), eval dim:  (2000, 4)


In [12]:
# Saving train and eval data
file_path = "/content/drive/MyDrive/data/"
train.to_csv(os.path.join(file_path, "train_subset.csv"), index=False)
eval.to_csv(os.path.join(file_path, "eval.csv"), index=False)

In [13]:
dataset = load_dataset('csv', data_files={'train': file_path + 'train_subset.csv', 'eval': file_path + 'eval.csv'}, encoding='ISO-8859-1')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e2368febdc8899dc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:

MODEL = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [15]:
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}
# Defining a function to tokenize text
def tokenize_data(example):
    return tokenizer(example['safe_text'], padding='max_length')

# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7999
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [17]:
#Load the pretrained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

*We have to shuffle the original dataset in order to minimise variance and ensure that the model will generalise well to new, unseen data points*

In [18]:
# reorganizing the data to help with overfitting
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['eval'].shuffle(seed=10)

In [19]:
#Specify training arguments
training_args = TrainingArguments(
                output_dir='./results', #output directory
                num_train_epochs=10,  #Total number of training epochs to perform
                per_device_train_batch_size=8, #Batch size for device during training
                per_device_eval_batch_size= 8, #Batch size for evaluation
                evaluation_strategy = 'epoch', #Evaluation is done at the end of each epoch
                eval_steps=100,
                save_strategy='epoch', #save at the end of each epoch
                save_steps=100,
                warmup_steps= 500, #Number of steps used for a linear warmup from 0 to learning_rate
                learning_rate = 5e-6, #learining rate
                seed=42,
                weight_decay = 0.01, # the weight decay value
                logging_strategy='epoch',
                logging_dir = './logs',
                logging_steps =100,
                load_best_model_at_end=True, #Whether or not to load the best model found during training at the end of training
                )

In [20]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [21]:
# Launch the learning process: training 
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.807600,0.668468
2,0.612900,0.624573
3,0.523300,0.656549
4,0.436000,0.694400
5,0.361300,0.786404
6,0.297000,0.896656
7,0.254600,1.011840
8,0.222100,1.098288
9,0.205100,1.154542
10,0.176600,1.191352


TrainOutput(global_step=10000, training_loss=0.38965528259277343, metrics={'train_runtime': 7735.2678, 'train_samples_per_second': 10.341, 'train_steps_per_second': 1.293, 'total_flos': 2.104644228406272e+16, 'train_loss': 0.38965528259277343, 'epoch': 10.0})

**Observation**

model seems to ovefit the training data as after the third epoch. Reducing the number of epochs to 2 or 3 might be helpful.

In [ ]:
training_args = TrainingArguments(
                output_dir='./results', #output directory
                num_train_epochs=3,  #Total number of training epochs to perform
                per_device_train_batch_size=8, #Batch size for device during training
                per_device_eval_batch_size= 8, #Batch size for evaluation
                evaluation_strategy = 'epoch', #Evaluation is done at the end of each epoch
                eval_steps=100,
                save_strategy='epoch', #save at the end of each epoch
                save_steps=100,
                warmup_steps= 500, #Number of steps used for a linear warmup from 0 to learning_rate
                learning_rate = 5e-6, #learining rate
                seed=42,
                weight_decay = 0.01, # the weight decay value
                logging_strategy='epoch',
                logging_dir = './logs',
                logging_steps =100,
                load_best_model_at_end=True, #Whether or not to load the best model found during training at the end of training
                )

In [22]:
from datasets import load_metric
import numpy as np
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    rmse = np.sqrt(np.mean((predictions - labels)**2))
    return {"rmse": rmse}

<ipython-input-22-e73aa7a42ef1>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [24]:
# Launch the final evaluation 
trainer.evaluate()

{'eval_loss': 0.62457275390625,
 'eval_rmse': 0.6946221994724903,
 'eval_runtime': 62.7269,
 'eval_samples_per_second': 31.884,
 'eval_steps_per_second': 3.986}